In [ ]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk

# Create a function to apply filters
def apply_filters():
    selected_filters = []

    if hue_saturation_var.get():
        hue_saturation_value = hue_saturation_scale.get() / 10.0
        selected_filters.append(lambda img: apply_hue_saturation(img, hue_saturation_value))
    if sharpening_var.get():
        sharpening_value = sharpening_scale.get()
        selected_filters.append(lambda img: apply_sharpening(img, sharpening_value))
    if gaussian_blur_var.get():
        gaussian_blur_value = gaussian_blur_scale.get()
        selected_filters.append(lambda img: apply_gaussian_blur(img, gaussian_blur_value))
    if edge_detection_var.get():
        selected_filters.append(apply_edge_detection)

    if not selected_filters:
        return

    image = cv2.imread(file_path)
    original_image = image.copy()  # Make a copy of the original image

    for filter_func in selected_filters:
        image = filter_func(image)

    cv2.imshow("Filtered Image", image)
    cv2.imshow("Original Image", original_image)

# Define filter functions with adjustable parameters
def apply_hue_saturation(image, value):
    # Apply Hue/Saturation adjustment
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv_image[:, :, 1] = hsv_image[:, :, 1] * value  # Adjust saturation
    return cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

def apply_sharpening(image, value):
    # Apply Sharpening filter
    kernel = np.array([[-1, -1, -1],
                      [-1, value, -1],  # Adjust sharpening value here
                      [-1, -1, -1]])
    return cv2.filter2D(image, -1, kernel)

def apply_gaussian_blur(image, value):
    # Apply Gaussian Blur filter
    kernel_size = int(2 * value + 1)  # Adjust kernel size based on the slider value
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

def apply_edge_detection(image):
    # Apply Edge Detection filter using Canny edge detector
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_image, 100, 200)  # You can adjust the thresholds as needed
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)  # Convert edges to a BGR image for display


# Create a function to open an image file
def open_image():
    global file_path
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp *.gif *.tiff")])
    if file_path:
        apply_button["state"] = "normal"

# Create the main GUI window
root = tk.Tk()
root.title("Image Filters")

# Create variables to track filter selections
hue_saturation_var = tk.BooleanVar()
sharpening_var = tk.BooleanVar()
gaussian_blur_var = tk.BooleanVar()
edge_detection_var = tk.BooleanVar()

# Create GUI elements
open_button = tk.Button(root, text="Open Image", command=open_image)
apply_button = tk.Button(root, text="Apply Filters", command=apply_filters, state="disabled")
hue_saturation_check = ttk.Checkbutton(root, text="Hue/Saturation", variable=hue_saturation_var)
sharpening_check = ttk.Checkbutton(root, text="Sharpening", variable=sharpening_var)
gaussian_blur_check = ttk.Checkbutton(root, text="Gaussian Blur", variable=gaussian_blur_var)
edge_detection_check = ttk.Checkbutton(root, text="Edge Detection", variable=edge_detection_var)

# Create scales (sliders) for adjustable parameters
hue_saturation_scale = ttk.Scale(root, from_=1, to=10, orient="horizontal", length=200)
sharpening_scale = ttk.Scale(root, from_=1, to=10, orient="horizontal", length=200)
gaussian_blur_scale = ttk.Scale(root, from_=1, to=10, orient="horizontal", length=200)

# Pack GUI elements
open_button.pack()
apply_button.pack()
hue_saturation_check.pack()
hue_saturation_scale.pack()
sharpening_check.pack()
sharpening_scale.pack()
gaussian_blur_check.pack()
gaussian_blur_scale.pack()
edge_detection_check.pack()

root.mainloop()
